In [1]:
from flask import Flask , request, render_template , make_response ,jsonify ## flask 라이브러리에서 Flask import
from flask_cors import CORS
import os
import time
from urllib.request import urlretrieve 
import pandas as pd
import numpy as np
import requests as req
import json
#예측 시스템용
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [ ]:
app = Flask(__name__)
CORS(app)
#행정구별 인구수
@app.route('/gu_dashboard',methods=['POST'])
def gu_dashboard():
    print('gu_dashbaord connect')
    if request.method == 'POST':
        gu_name = request.get_data(as_text=True)
        #gu_name = request.form.get('testdata') #구 이름 받는 값
            
        # 구 인구 수 데이터 불러오기 / 전처리
        pop_dashboard=pd.read_csv('./data/pop_dashboard.csv')
        mask=pop_dashboard['행정구']==gu_name
        pop=pop_dashboard[mask]
        pop=pop[['총인구수','내국인','외국인']].astype(int)
            
        #유흥시설 현황 데이터 불러오기 / 전처리
        uh_dashboard=pd.read_csv('./data/uh_dashboard.csv')
        uh_dashboard=uh_dashboard.set_index('업태구분명',drop=True, append=False, inplace=False, verify_integrity=False)
          
        #주거형태 현황 데이터 불러오기 / 전처리
        housingtype_dashboard=pd.read_csv('./data/housingType_dashboard.csv')
        housingtype_dashboard.sort_values(['행정구'],inplace=True,ignore_index=True)
        mask_h=housingtype_dashboard['행정구']==gu_name
        h_dashboard=housingtype_dashboard[mask_h]
                        
                                                
        #####그래프 입력 값
        b_x_list=[] #라벨 업태구분명
        b_y_list=[] #빈도, 업태 수
        for i in uh_dashboard.index : #cnt 0인 업태명은 제외하고자 함
            if uh_dashboard[gu_name][i] > 0 :
                b_x_list.append(i)
                b_y_list.append(int(uh_dashboard[gu_name][i]))
                    
            

        p_x=pop.columns.to_list() #라벨 데이터
        p_y=pop.iloc[0].to_list() #인구수
           
        h_x=h_dashboard.columns[2:].to_list() #주거형태
        h_y=h_dashboard.values[0][2:].tolist() #주거형태별 갯수

        data = {
          'pop_label':p_x, #pop_label
          'pop_cnt':p_y, #pop_cnt
          'business_name': b_x_list, #업태구분명 라벨
          'business_cnt': b_y_list, #업태 수
          'house_type': h_x,#주거형태 라벨
          'house_cnt': h_y #각 주거형태 개수
        }
        
        print(jsonify(data))
        response = make_response(jsonify(data))
        response.headers.add("Access-Control-Allow-Origin", "*")
        return response

#서울시 범죄/검거율   
@app.route('/seoul_dashboard', methods=['POST'])
def seoul_dashboard():
    print('seoul_dashboard connect')
    # 서울시 범죄 데이터전처리
    crime = pd.read_csv('data/seoul_crime.csv', skiprows=2)
    crime.columns = ['행정구별', '총범죄발생', '총범죄검거', '살인발생', '살인검거', '강도발생', '강도검거', '강간강제추행발생', '강간강제추행검거', '절도발생', '절도검거', '폭력발생', '폭력검거']
    crime['살인검거율'] = crime['살인검거'] / crime['살인발생'] * 100
    crime['강도검거율'] = crime['강도검거'] / crime['강도발생'] * 100
    crime['강간강제추행검거율'] = crime['강간강제추행검거'] / crime['강간강제추행발생'] * 100
    crime['절도검거율'] = crime['절도검거'] / crime['절도발생'] * 100
    crime['폭력검거율'] = crime['폭력검거'] / crime['폭력발생'] * 100
    crime_arr = crime[['행정구별', '강도검거', '강간강제추행검거', '살인검거', '절도검거', '폭력검거']]
    crime_per = crime[['행정구별', '강도검거율', '강간강제추행검거율', '살인검거율', '절도검거율', '폭력검거율']]
    crime_occ = crime[['행정구별','살인발생', '강도발생','강간강제추행발생', '절도발생', '폭력발생']]
    categories = ['살인건수', '강도건수', '강간강제추행건수', '절도건수', '폭력건수']
    
     #년도 기준 17개 시도 데이터전처리
    na_pop = pd.read_csv('data/na_pop.csv',encoding='cp949',skiprows=2)
    na_pop.columns = ['전국','총인구수','남자인구수','여자인구수']    
    na_crime = pd.read_csv('data/na_crime.csv')    
    na_pop_cri = pd.concat([na_pop,na_crime],axis=1)    
    na_pop_cri['천명당건수']=na_pop_cri['발생건수']/(na_pop_cri['총인구수']/1000)    
    police = pd.read_csv('data/police.csv')    
    result = police.groupby('시도청')['경찰관'].sum()   
    order_of_locations = ['서울','부산','대구','인천','광주','대전','울산','세종','경기','강원','충북','충남','전북','전남','경북','경남','제주']    
    result = result.reindex(order_of_locations)    
    result = result.reset_index()    
    all_crime = pd.concat([na_pop_cri,result],axis=1)    
    all_crime['경찰관한명당범죄건수']=all_crime['발생건수']/all_crime['경찰관']
    
    # 서울시 5대 범죄 검거 건수
    top_group_sizes = crime_arr.iloc[0, 1:].values    
    top_group_sizes_list=[]
    for j in top_group_sizes:
        top_group_sizes_list.append(int(j))
    
    # 서울시 5대 범죄 검거율
    subgroup_sizes = crime_per.iloc[0, 1:].values
    subgroup_sizes_list=[]
    for i in subgroup_sizes:
        subgroup_sizes_list.append(int(i))
        
    #서울시 5대 범죄 발생 건수
    values = crime_occ.iloc[0, 1:].values
    values_list=[]
    for i in values:
        values_list.append(int(i))
    
    crime_all = all_crime.iloc[:, 5].values
    # 전국 시도별 범죄 발생 건수
    a_list=[]
    for a in crime_all:
        a_list.append(int(a))
    
    pol_cri = all_crime.iloc[:, 10].values
    b_list=[]
    for b in pol_cri:
        b_list.append(int(b))
    
    info_cri = all_crime.iloc[:, 7].values
    c_list=[]
    for c in info_cri:
        c_list.append(int(c))
    
    if request.method == 'POST':
        data = {
            'top_group_sizes': top_group_sizes_list, #5대 범죄 건수
            'subgroup_sizes': subgroup_sizes_list, #5대 범죄 검거율
            'categories': categories, #범죄종류
            'values' : values_list, #해당 범죄 건수
            'crime' : a_list, #년도 기준 17개 시도 범죄 건수
            'pol_cri': b_list, #년도 기준 17개 시도 경찰 명당 범죄 건수
            'info_cri' : c_list #년도 기준 17개 시도 인구 천명당 범죄 건수
        }
        response = make_response(jsonify(data))
        print(jsonify(data))
        response.headers.add("Access-Control-Allow-Origin", "*")
        return response
    
#예측 시스템용
@app.route('/pre', methods=['POST'])
def pre():
    print("predict connect")
    
    #구이름 데이터 받아오기
    gu_name = request.get_data(as_text=True)
    
    # 서울시 5대 범죄 데이터
    seoul_crime = pd.read_csv('predict_data/seoul_crime.csv', skiprows=3)
    seoul_crime.columns = ['행정구','총발생','총검거','살인발생','살인검거','강도발생','강도검거','강간강제추행발생', '강간강제추행검거','절도발생','절도검거','폭력발생','폭력검거']

    # 서울시 인구수 데이터
    seo_pop = pd.read_csv('predict_data/seo_pop.csv', skiprows=3)
    seo_pop.columns = ['합계','행정구','행정동','전체인구수','남자인구수','여자인구수','한국인구수','한국남자','한국여자','외국인구수','외국남자','외국여자']

    # 서울시 각구별 경찰서
    seoul_police_office = pd.read_csv('predict_data/seoul_polic.csv')

    # 서울시 cctv
    seoul_cctv = pd.read_csv('predict_data/seoul_cctv.csv', encoding='cp949')
    
    seo_gu = seoul_crime['행정구'].unique()
        
    # 각 구별 인구수
    pop_list = []
    for gu in seo_gu:
        pop_gu = seo_pop[seo_pop['행정구']==gu]['전체인구수'].sum()/2
        pop_list.append(pop_gu)
    seoul_crime['구별인구수'] = pd.Series(pop_list)
    
    # 각 구별 경찰서 수
    pof_list=[]
    for gu in seo_gu:
        pof_gu = seoul_police_office[seoul_police_office['행정구']==gu]['경찰서'].count()
        pof_list.append(pof_gu)
    seoul_crime['구별경찰서수'] = pd.Series(pof_list)
    
    # 각 구별 cctv 수
    cctv_list = []
    for gu in seo_gu:
        cctv_gu = seoul_cctv[seoul_cctv['자치구']==gu]['CCTV 수량'].sum()
        cctv_list.append(cctv_gu)
    seoul_crime['구별cctv수'] = pd.Series(cctv_list)
    
    seoul_crime['안전지수'] = (
        0.4 * seoul_crime['총검거'] / seoul_crime['총발생'] +
        0.3 * seoul_crime['구별인구수'] / seoul_crime['총발생'] +
        0.1 * seoul_crime['구별경찰서수'] / seoul_crime['구별인구수'] +
        0.2 * seoul_crime['구별cctv수'] / seoul_crime['구별인구수'])
    
    
    features = seoul_crime[['총발생', '총검거', '구별인구수', '구별경찰서수', '구별cctv수']]
    target = seoul_crime['안전지수']
    
    scaler = StandardScaler()   
    features_scaled = scaler.fit_transform(features)
    
    X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)
    
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    predictions = model.predict(X_test)
    
    all_predictions = model.predict(features_scaled)
    
    mse_test = mean_squared_error(y_test, predictions)
    print(f'평균제곱오차 (test): {mse_test}')
    
    mse_all = mean_squared_error(target, all_predictions)
    print(f'평균제곱오차 (all): {mse_all}')
    
    result_df_all = pd.DataFrame({'실제값': target, '예측값': all_predictions})
    print(result_df_all)
    
    df2=pd.DataFrame(all_predictions)
    df1=pd.DataFrame(['종로구', '중구', '용산구', '성동구', '광진구', 
                      '동대문구', '중랑구', '성북구', '강북구', '도봉구',
                      '노원구', '은평구', '서대문구', '마포구', '양천구',
                      '강서구', '구로구', '금천구', '영등포구', '동작구',
                      '관악구', '서초구', '강남구', '송파구', '강동구'])
    
    features=pd.concat([df1,features],axis=1)
    features.columns=['행정구', '총발생', '총검거', '구별인구수', '구별경찰서수', '구별cctv수']
    
    mask=features['행정구']==gu_name
    gu_crime_cnt=int(features[mask]['총발생'])
    gu_popBy_cnt=int(features[mask]['총발생']/(features[mask]['구별인구수']/1000))
    gu_PolBy_cnt=int(features[mask]['총발생']/features[mask]['구별경찰서수'])
    
    
    all_predictions=pd.concat([df1,df2],axis=1)
    all_predictions.columns=['행정구','예측값']
    
    mask=all_predictions['행정구']==gu_name
    pre_data=int(all_predictions[mask]['예측값'])
    

    if request.method == 'POST':
        data = {
            "predict_data": pre_data, #해당 구 안전지수 예측도
            "gu_crime_cnt": gu_crime_cnt, #해당 구 범죄 건수
            "gu_popBy_cnt": gu_popBy_cnt, #해당 구 인구 천명당 범죄 건수
            "gu_PolBy_cnt": gu_PolBy_cnt #해당 구 내 경찰 명당 범죄 건수
               }
        print(data)
        response = make_response(jsonify(data))
        print(jsonify(data))
        response.headers.add("Access-Control-Allow-Origin", "*")
        return response


if __name__ == "__main__":
    app.run(host='119.200.31.33',port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://119.200.31.33:5000
Press CTRL+C to quit
218.149.140.106 - - [01/Dec/2023 16:48:15] "POST /seoul_dashboard HTTP/1.1" 200 -


seoul_dashboard connect
<Response 495 bytes [200 OK]>


205.210.31.231 - - [01/Dec/2023 16:59:48] "GET / HTTP/1.1" 404 -
167.248.133.33 - - [01/Dec/2023 18:54:27] "GET / HTTP/1.1" 404 -
167.248.133.33 - - [01/Dec/2023 18:54:28] code 505, message Invalid HTTP version (2.0)
167.248.133.33 - - [01/Dec/2023 18:54:28] "PRI * HTTP/2.0" 505 -
27.0.238.186 - - [01/Dec/2023 22:41:09] "GET / HTTP/1.0" 404 -
211.231.103.92 - - [01/Dec/2023 22:41:09] "GET /pre HTTP/1.0" 405 -
27.0.238.70 - - [01/Dec/2023 22:41:09] "GET /seoul_dashboard HTTP/1.0" 405 -
198.235.24.32 - - [02/Dec/2023 04:06:55] "GET / HTTP/1.0" 404 -
205.210.31.186 - - [02/Dec/2023 13:17:24] "GET / HTTP/1.1" 404 -
162.142.125.10 - - [02/Dec/2023 20:43:13] code 400, message Bad request version ('À\x14À')
162.142.125.10 - - [02/Dec/2023 20:43:13] " î  ê²Z)ïÚfosbÑLÚ¥Â`¬»Øªí^áQÑ÷ xñHQìzïEÜ½Ç¨¦Ú²¬Øú &Ì¨Ì©À/À0À+À,ÀÀ	ÀÀ" 400 -
162.142.125.10 - - [02/Dec/2023 20:43:16] "GET / HTTP/1.1" 404 -
162.142.125.10 - - [02/Dec/2023 20:43:20] "GET / HTTP/1.1" 404 -
162.142.125.